# Day 4 - Advent of Code 2020

### Part 1

First we need to load in our data. We are just going to use a txt file for this

In [15]:
fh = open('data.txt', 'r')

Load the data into an array

In [16]:
data = [x for x in fh.readlines()]

In [17]:
data[1:10]

['pid:221225902 cid:61 hgt:186cm eyr:2021 hcl:#7d3b0c\n',
 '\n',
 'hcl:#efcc98 hgt:178 pid:433543520\n',
 'eyr:2020 byr:1926\n',
 'ecl:blu cid:92\n',
 'iyr:2010\n',
 '\n',
 'iyr:2018\n',
 'eyr:2026\n']

So in order to work with this data, we need to create each passport record. In the current format, multiple elements of the array represent a singel record, we need to merge them so that one element represent all the information we have for that passport

In [18]:
from typing import List
def merge_passport_data(data:List[int]):
    current_passport = ''
    all_passports = []
    for (idx, x) in enumerate(data):
        if x == '\n':
            all_passports.append(current_passport)
            current_passport = ''
            
        else:
            current_passport += " " + x.strip('\n') + ' '
    return all_passports
            

In [19]:
passports  = merge_passport_data(data)

In [20]:
passports[0]

' ecl:hzl byr:1926 iyr:2010  pid:221225902 cid:61 hgt:186cm eyr:2021 hcl:#7d3b0c '

Now we need to write a validate function that will make sure that all the keys exist, this can be done simply as follows

In [21]:
def validate(passport) -> bool:
    mandatory_fields = ['byr','iyr','eyr', 'hgt', 'hcl' ,'ecl','pid']
    for field in mandatory_fields:
        if field not in passport:
            return False
    return True

In [22]:
valid_passport = [validate(x) for x in passports]

In [23]:
sum(valid_passport)

237

### Part 2

Okay, i was too lazy in the part 1 and now i am paying for it. What i should have done was create a data model that i could map each passport into. This would have made part 2 easy, guess i have to do it now. To do this i will just use a simpe data class

In [24]:
from dataclasses import dataclass

@dataclass
class Passport:
    def __getitem__(cls, x):
        return getattr(cls, x)
    byr: int
    iyr: int
    eyr: int
    hgt: str
    hcl: str
    ecl: str
    pid: str
    cid: str

In [47]:
import re
def get_text_after_regex(element_key, data_string):
    regex = f' {element_key}:(.+?) .*'
    matchObject = re.search(regex, data_string)
    if matchObject:
        return matchObject.group(1)
    else:
        return None


def generate_passport(passport_string):
    fields = ['byr','iyr','eyr', 'hgt', 'hcl' ,'ecl','pid', 'cid']
    parsed_data = {k:get_text_after_regex(k, passport_string) for k in fields}
    return Passport(**parsed_data)



def validate_passport(passport: Passport) -> bool:
#     Make sure all fields exist
    mandatory_fields = ['byr','iyr','eyr', 'hgt', 'hcl' ,'ecl','pid']
    for field in mandatory_fields:
        if passport[field] == None:
            print(field)
            return False
#     byr (Birth Year) - four digits; at least 1920 and at most 2002.
    if len(passport.byr) != 4 and not (1920 < passport.byr < 2002):
        return False
    if len(passport.iyr) != 4 and not (2010 < passport.byr < 2020):
        return False
    if len(passport.eyr) != 4 and not (2020 < passport.byr < 2030):
        return False
    if 'cm' in passport.hgt:
        if not (150 < int(passport.hgt.strip('cm')) < 193):
            return False
    
    if 'in' in passport.hgt:
        if not (59 < int(passport.hgt.strip('in')) < 76):
            return False
    if not re.search(r'^#(?:[0-9a-fA-F]{3}){1,2}$', passport.hcl):
        return False
    if passport.ecl not in ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']:
        return False
    if len(passport.pid) != 9:
        return False
    return True
      

In [48]:
valid_with_rules = [validate_passport(generate_passport(passport_string=x)) for x in passports]

hgt
byr
pid
byr
eyr
byr
eyr
iyr
hcl
eyr
ecl
pid
hcl
eyr
iyr
hcl
ecl
hcl
byr
byr
eyr
eyr
pid
iyr
hgt
ecl
pid
ecl
ecl
iyr
eyr
byr
iyr
ecl
pid
hgt
ecl
byr
hgt
hcl
ecl
pid
iyr
ecl
byr
hgt
eyr
eyr
hcl
eyr
hcl
byr


In [50]:
sum(valid_with_rules)

172